🤔 Why do Agents need Tools?¶
The Problem

Without tools, the agent's knowledge is frozen in time — it can't access today's news or your company's inventory. It has no connection to the outside world, so the agent can't take actions for you.

The Solution: Tools are what transform your isolated LLM into a capable agent that can actually help you get things done.

In this notebook, you'll:

✅ Turn your Python functions into Agent tools
✅ Build an Agent and use it as a tool in another agent
✅ Build your first multi-tool agent
✅ Explore the different tool types in ADK

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip uninstall -y google-adk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U google-adk


  Obtaining dependency information for google-adk from https://files.pythonhosted.org/packages/65/30/1012b25cb0dfb630c2b8c040c181f0b6559efb00d1e6007c4eca24970fb2/google_adk-1.18.0-py3-none-any.whl.metadata
  Using cached google_adk-1.18.0-py3-none-any.whl.metadata (13 kB)
Using cached google_adk-1.18.0-py3-none-any.whl (2.2 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [5]:
from google.adk.agents import LlmAgent

In [6]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool
from google.genai import types


In [7]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

2.1: Building Custom Function Tools
Example: Currency Converter Agent
This agent can convert currency from one denomination to another and calculates the fees to do the conversion. The agent has two custom tools and follows the workflow:

Fee Lookup Tool - Finds transaction fees for the conversion (mock)
Exchange Rate Tool - Gets currency conversion rates (mock)
Calculation Step - Calculates the total conversion cost including the fees

🤔 2.2: How to define a Tool?
Any Python function can become an agent tool by following these simple guidelines:

Create a Python function
Follow the best practices listed below
Add your function to the agent's tools=[] list and ADK handles the rest automatically.
🏆 ADK Best Practices in Action
Notice how our tools follow ADK best practices:

1. Dictionary Returns: Tools return {"status": "success", "data": ...} or {"status": "error", "error_message": ...}
2. Clear Docstrings: LLMs use docstrings to understand when and how to use tools
3. Type Hints: Enable ADK to generate proper schemas (str, dict, etc.)
4. Error Handling: Structured error responses help LLMs handle failures gracefully

These patterns make your tools reliable and easy for LLMs to use correctly.

In [8]:
def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """
    # This simulates looking up a company's internal fee structure.
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"Payment method '{method}' not found",
        }


print("✅ Fee lookup function created")
print(f"💳 Test: {get_fee_for_payment_method('platinum credit card')}")

✅ Fee lookup function created
💳 Test: {'status': 'success', 'fee_percentage': 0.02}


In [9]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

    # Static data simulating a live exchange rate API
    # In production, this would call something like: requests.get("api.exchangerates.com")
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

    # Input validation and processing
    base = base_currency.lower()
    target = target_currency.lower()

    # Return structured result with status
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}",
        }


print("Exchange rate function created")
print(f"Test: {get_exchange_rate('USD', 'EUR')}")


Exchange rate function created
Test: {'status': 'success', 'rate': 0.93}


In [10]:
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_fee_for_payment_method, get_exchange_rate],
)

print("Currency agent created with custom function tools")
print("Available tools:")
print("• get_fee_for_payment_method - Looks up company fee structure")
print("• get_exchange_rate - Gets current exchange rates")

Currency agent created with custom function tools
Available tools:
• get_fee_for_payment_method - Looks up company fee structure
• get_exchange_rate - Gets current exchange rates


In [11]:
currency_runner = InMemoryRunner(agent=currency_agent)
_ = await currency_runner.run_debug(
    "I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?"
)


 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?


currency_agent > You will receive 455.70 Euros.

This is based on the following calculations:
1. The transaction fee for using a platinum credit card is 2.00%.
2. The fee amount is 10.00 USD (500 USD * 0.02).
3. The remaining amount after the fee is 490.00 USD (500 USD - 10 USD).
4. The exchange rate from USD to EUR is 0.93.
5. The final converted amount is 455.70 EUR (490.00 USD * 0.93).


The agent's instruction says "calculate the final amount after fees" but LLMs aren't always reliable at math. They might make calculation errors or use inconsistent formulas.

💡 Solution: Let's ask our agent to generate a Python code to do the math, and run it to give us the final result! Code execution is much more reliable than having the LLM try to do math in its head!
ADK has a built-in Code Executor capable of running code in a sandbox. Note: This uses Gemini's Code Execution capability.

Let's create a calculation_agent which takes in a Python code and uses the BuiltInCodeExecutor to run it.

In [12]:
pip install --upgrade google-adk


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from google.adk.tools import BuiltInCodeExecutor


ImportError: cannot import name 'BuiltInCodeExecutor' from 'google.adk.tools' (c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\adk\tools\__init__.py)

In [ ]:
pip show google-adk

Name: google-adk
Version: 1.18.0
Summary: Agent Development Kit
Home-page: 
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, typing-extensions, tzlocal, uvicorn, watchdog, websockets
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities
)

NameError: name 'BuiltInCodeExecutor' is not defined

In [ ]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

NameError: name 'calculation_agent' is not defined

In [ ]:
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

NameError: name 'enhanced_currency_agent' is not defined

In [ ]:
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)

In [ ]:
show_python_code_and_result(response)


1. Custom Tools
What: Tools you build yourself for specific needs

Advantage: Complete control over functionality — you build exactly what your agent needs

Function Tools ✅ (You've used these!)
What: Python functions converted to agent tools
Examples: get_fee_for_payment_method, get_exchange_rate
Advantage: Turn any Python function into an agent tool instantly
Long Running Function Tools
What: Functions for operations that take significant time
Examples: Human-in-the-loop approvals, file processing
Advantage: Agents can start tasks and continue with other work while waiting
Agent Tools ✅ (You've used these!)
What: Other agents used as tools
Examples: AgentTool(agent=calculation_agent)
Advantage: Build specialist agents and reuse them across different systems
MCP Tools
What: Tools from Model Context Protocol servers
Examples: Filesystem access, Google Maps, databases
Advantage: Connect to any MCP-compatible service without custom integration
OpenAPI Tools
What: Tools automatically generated from API specifications
Examples: REST API endpoints become callable tools
Advantage: No manual coding — just provide an API spec and get working tools
2. Built-in Tools
Built-in Tools

What: Pre-built tools provided by ADK

Advantage: No development time — use immediately with zero setup

Gemini Tools ✅ (You've used these!)
What: Tools that leverage Gemini's capabilities
Examples: google_search, BuiltInCodeExecutor
Advantage: Reliable, tested tools that work out of the box
Google Cloud Tools [needs Google Cloud access]
What: Tools for Google Cloud services and enterprise integration
Examples: BigQueryToolset, SpannerToolset, APIHubToolset
Advantage: Enterprise-grade database and API access with built-in security
Third-party Tools
What: Wrappers for existing tool ecosystems
Examples: Hugging Face, Firecrawl, GitHub Tools
Advantage: Reuse existing tool investments — no need to rebuild what already exists


2B
🚀 Agent Tool Patterns and Best Practices¶
Welcome to Day-2 of the Kaggle 5-day Agents course!

In the previous notebook, you learned how to add custom Python functions as tools to your agent. In this notebook, we'll take the next step: consuming external MCP services and handling long-running operations.

In this notebook, you'll learn how to:

✅ Connect to external MCP servers
✅ Implement long-running operations that can pause agent execution for external input
✅ Build resumable workflows that maintain state across conversation breaks
✅ Understand when and how to use these patterns

Model Context Protocol¶
So far, you have learned how to create custom functions for your agents. But connecting to external systems (GitHub, databases, Slack) requires writing and maintaining API clients.

Model Context Protocol (MCP) is an open standard that lets agents use community-built integrations. Instead of writing your own integrations and API clients, just connect to an existing MCP server.

MCP enables agents to:

✅ Access live, external data from databases, APIs, and services without custom integration code
✅ Leverage community-built tools with standardized interfaces
✅ Scale capabilities by connecting to multiple specialized servers

2.1: How MCP Works
MCP connects your agent (the client) to external MCP servers that provide tools:

MCP Server: Provides specific tools (like image generation, database access)
MCP Client: Your agent that uses those tools
All servers work the same way - standardized interface
Architecture:

┌──────────────────┐
│   Your Agent     │
│   (MCP Client)   │
└────────┬─────────┘
         │
         │ Standard MCP Protocol
         │
    ┌────┴────┬────────┬────────┐
    │         │        │        │
    ▼         ▼        ▼        ▼
┌────────┐ ┌─────┐ ┌──────┐ ┌─────┐
│ GitHub │ │Slack│ │ Maps │ │ ... │
│ Server │ │ MCP │ │ MCP  │ │     │
└────────┘ └─────┘ └──────┘ └─────┘
2.2: Using MCP with Your Agent
The workflow is simple:

Choose an MCP Server and tool
Create the MCP Toolset (configure connection)
Add it to your agent
Run and test the agent
Step 1: Choose MCP Server

For this demo, we'll use the Everything MCP Server - an npm package (@modelcontextprotocol/server-everything) designed for testing MCP integrations.

It provides a getTinyImage tool that returns a simple test image (16x16 pixels, Base64-encoded). Find more servers: modelcontextprotocol.io/examples

‼️ NOTE: This is a demo server to learn MCP. In production, you'll use servers for Google Maps, Slack, Discord, etc.

Step 2: Create the MCP Toolset

The McpToolset is used to integrate an ADK Agent with an MCP Server.

What the code does:

Uses npx (Node package runner) to run the MCP server
Connects to @modelcontextprotocol/server-everything
Filters to only use the getTinyImage tool (the server has others, but we only need this one)

In [ ]:
from google.adk.tools.mcp import McpToolset
from google.adk.tools.mcp import StdioConnectionParams, StdioServerParameters

ModuleNotFoundError: No module named 'google.adk.tools.mcp'

In [ ]:
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

NameError: name 'McpToolset' is not defined

Step 3: Add MCP tool to agent

Let's add the mcp_server to the agent's tool array and update the agent's instructions to handle requests to generate tiny images.

In [ ]:
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="Use the MCP Tool to generate images for user queries",
    tools=[mcp_image_server],
)

NameError: name 'mcp_image_server' is not defined

Step 4: Test the agent

Ask the agent to generate an image. Watch it use the MCP tool:

In [ ]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)
response = await runner.run_debug("Provide a sample tiny image", verbose=True)

Display the image:

The server returns base64-encoded image data. Let's decode and display it:

In [ ]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))



🔄 Section 3: Long-Running Operations (Human-in-the-Loop)
So far, all tools execute and return immediately:

User asks → Agent calls tool → Tool returns result → Agent responds

But what if your tools are long-running or you need human approval before completing an action?

Example: A shipping agent should ask for approval before placing a large order.

User asks → Agent calls tool → Tool PAUSES and asks human → Human approves → Tool completes → Agent responds

This is called a Long-Running Operation (LRO) - the tool needs to pause, wait for external input (human approval), then resume.

When to use Long-Running Operations:

💰 Financial transactions requiring approval (transfers, purchases)
🗑️ Bulk operations (delete 1000 records - confirm first!)
📋 Compliance checkpoints (regulatory approval needed)
💸 High-cost actions (spin up 50 servers - are you sure?)
⚠️ Irreversible operations (permanently delete account)

3.1: What We're Building Today¶
Let's build a shipping coordinator agent with one tool that:

Auto-approves small orders (≤5 containers)
Pauses and asks for approval on large orders (>5 containers)
Completes or cancels based on the approval decision
This demonstrates the core long-running operation pattern: pause → wait for human input → resume.

1. Request approval: Call tool_context.request_confirmation()
2. Check approval status: Read tool_context.tool_confirmation

In [14]:
LARGE_ORDER_THRESHOLD = 5


def place_shipping_order(
    num_containers: int, destination: str, tool_context: ToolContext
) -> dict:
    """Places a shipping order. Requires approval if ordering more than 5 containers (LARGE_ORDER_THRESHOLD).

    Args:
        num_containers: Number of containers to ship
        destination: Shipping destination

    Returns:
        Dictionary with order status
    """

 # SCENARIO 1: Small orders (≤5 containers) auto-approve
    if num_containers <= LARGE_ORDER_THRESHOLD:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-AUTO",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order auto-approved: {num_containers} containers to {destination}",
        }
# SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Large order: {num_containers} containers to {destination}. Do you want to approve?",
            payload={"num_containers": num_containers, "destination": destination},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Order for {num_containers} containers requires approval",
        }
# SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-HUMAN",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order approved: {num_containers} containers to {destination}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Order rejected: {num_containers} containers to {destination}",
        }

NameError: name 'ToolContext' is not defined

How the Three Scenarios Work¶
The tool handles three scenarios by checking tool_context.tool_confirmation:

Scenario 1: Small order (≤5 containers): Returns immediately with auto-approved status.

tool_context.tool_confirmation is never checked
Scenario 2: Large order - FIRST CALL

Tool detects it's a first call: if not tool_context.tool_confirmation:
Calls request_confirmation() to request human approval
Returns {'status': 'pending', ...} immediately
ADK automatically creates adk_request_confirmation event
Agent execution pauses - waiting for human decision
Scenario 3: Large order - RESUMED CALL

Tool detects it's resuming: if not tool_context.tool_confirmation: is now False
Checks human decision: tool_context.tool_confirmation.confirmed
If True → Returns approved status
If False → Returns rejected status
Key insight: Between the two calls, your workflow code (in Section 4) must detect the adk_request_confirmation event and resume with the approval decision.

3.4: Create the Agent, App and Runner¶
Step 1: Create the agent

Add the tool to the Agent. The tool decides internally when to request approval based on the order size.

In [15]:
shipping_agent = LlmAgent(
    name="shipping_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a shipping coordinator assistant.
  
  When users request to ship containers:
   1. Use the place_shipping_order tool with the number of containers and destination
   2. If the order status is 'pending', inform the user that approval is required
   3. After receiving the final result, provide a clear summary including:
      - Order status (approved/rejected)
      - Order ID (if available)
      - Number of containers and destination
   4. Keep responses concise but informative
  """,
    tools=[FunctionTool(func=place_shipping_order)],
)
 

NameError: name 'FunctionTool' is not defined

Step 2: Wrap in resumable App

The problem: A regular LlmAgent is stateless - each call is independent with no memory of previous interactions. If a tool requests approval, the agent can't remember what it was doing.

The solution: Wrap your agent in an App with resumability enabled. The App adds a persistence layer that saves and restores state.

What gets saved when a tool pauses:

All conversation messages so far
Which tool was called (place_shipping_order)
Tool parameters (10 containers, Rotterdam)
Where exactly it paused (waiting for approval)

In [ ]:
shipping_app = App(
    name="shipping_coordinator",
    root_agent=shipping_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

Step 3: Create Session and Runner with the App

Pass app=shipping_app instead of agent=... so the runner knows about resumability.

In [ ]:
session_service = InMemorySessionService()

# Create runner with the resumable app
shipping_runner = Runner(
    app=shipping_app,  # Pass the app instead of the agent
    session_service=session_service,
)


✅ Recap: Your pausable shipping agent is now complete!

You've created:

✅ A tool that can pause for approval (place_shipping_order)
✅ An agent that uses this tool (shipping_agent)
✅ A resumable app that saves state (shipping_app)
✅ A runner that can handle pause/resume (shipping_runner)
Next step: Build the workflow code and test that our Agent detects pauses and handles approvals.



🏗️ Section 4: Building the Workflow
‼️ Important: The workflow code uses ADK concepts like Sessions, Runners, and Events. We'll cover what you need to know for long-running operations in this notebook. For deeper understanding, we will cover these topics in Day 3, or you can check out the ADK docs and this video.

4.1: ⚠️ The Critical Part - Handling Events in Your Workflow
The agent won't automatically handle pause/resume. Every long-running operation workflow requires you to:

Detect the pause: Check if events contain adk_request_confirmation
Get human decision: In production, show UI and wait for user click. Here, we simulate it.
Resume the agent: Send the decision back with the saved invocation_id
4.2 Understand Key Technical Concepts
👉 events - ADK creates events as the agent executes. Tool calls, model responses, function results - all become events

👉 adk_request_confirmation event - This event is special - it signals "pause here!"

Automatically created by ADK when your tool calls request_confirmation()
Contains the invocation_id
Your workflow must detect this event to know the agent paused
👉 invocation_id - Every call to run_async() gets a unique invocation_id (like "abc123")

When a tool pauses, you save this ID
When resuming, pass the same ID so ADK knows which execution to continue
Without it, ADK would start a NEW execution instead of resuming the paused one
4.3: Helper Functions to Process Events
These handle the event iteration logic for you.

check_for_approval() - Detects if the agent paused

Loops through all events and looks for the special adk_request_confirmation event
Returns approval_id (identifies this specific request) and invocation_id (identifies which execution to resume)
Returns None if no pause detected


In [ ]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None
print_agent_response() - Displays agent text


In [ ]:

<!-- Simple helper to extract and print text from events -->
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")
create_approval_response() - Formats the human decision



Takes the approval info and boolean decision (True/False) from the human
Creates a FunctionResponse that ADK understands
Wraps it in a Content object to send back to the agent


In [ ]:
def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )



4.4: The Workflow Function - Let's tie it all together!
The run_shipping_workflow() function orchestrates the entire approval flow.

Look for the code explanation in the cell below.

Long-running operation workflow


In [ ]:
async def run_shipping_workflow(query: str, auto_approve: bool = True):
    """Runs a shipping workflow with approval handling.

    Args:
        query: User's shipping request
        auto_approve: Whether to auto-approve large orders (simulates human decision)
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name="shipping_coordinator", user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_containers > 5, the Agent returns the special `adk_request_confirmation` event
    async for event in shipping_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    approval_info = check_for_approval(events)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in shipping_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
        print_agent_response(events)

    print(f"{'='*60}\n")




Code breakdown
Step 1: Send initial request to the Agent

Call run_async() to start agent execution
Collect all events in a list for inspection
Step 2: Detect Pause

Call check_for_approval(events) to look for the special event: adk_request_confirmation
Returns approval info (with invocation_id) if the special event is present; None if completed
Step 3: Resume execution

PATH A:

If the approval info is present, at this point the Agent pauses for human input.
Once the Human input is available, call the agent again using run_async() and pass in the Human input.
Critical: Same invocation_id (tells ADK to RESUME, not restart)
Display agent's final response after resuming
PATH B:

If the approval info is not present, then approval is not needed and the agent completes execution.
🎬 4.5: Demo: Testing the Workflow
Now, let's run our demos. Notice how much cleaner and easier to read they are. All the complex logic for pausing and resuming is now hidden away in our run_workflow helper function, allowing us to focus on the tasks we want the agent to perform.

Note: You may see warnings like Warning: there are non-text parts in the response: ['function_call'] - this is normal and can be ignored. It just means the agent is calling tools in addition to generating text.

============================================================

Detailed timeline:

Here's what happens step-by-step when you run run_shipping_workflow("Ship 10 containers to Rotterdam", auto_approve=True):

TIME 1: User sends "Ship 10 containers to Rotterdam"
        ↓
TIME 2: Workflow calls shipping_runner.run_async(...)
        ADK assigns a unique invocation_id = "abc123"
        ↓
TIME 3: Agent receives user message, decides to use place_shipping_order tool
        ↓
TIME 4: ADK calls place_shipping_order(10, "Rotterdam", tool_context)
        ↓
TIME 5: Tool checks: num_containers (10) > 5
        Tool calls tool_context.request_confirmation(...)
        ↓
TIME 6: Tool returns {'status': 'pending', ...}
        ↓
TIME 7: ADK creates adk_request_confirmation event with invocation_id="abc123"
        ↓
TIME 8: Workflow detects the event via check_for_approval()
        Saves approval_id and invocation_id="abc123"
        ↓
TIME 9: Workflow gets human decision → True (approve)
        ↓
TIME 10: Workflow calls shipping_runner.run_async(..., invocation_id="abc123")
         Passes approval decision as FunctionResponse
         ↓
TIME 11: ADK sees invocation_id="abc123" - knows to RESUME (instead of starting new)
         Loads saved state from TIME 7
         ↓
TIME 12: ADK calls place_shipping_order again with same parameters
         But now tool_context.tool_confirmation.confirmed = True
         ↓
TIME 13: Tool returns {'status': 'approved', 'order_id': 'ORD-10-HUMAN', ...}
         ↓
TIME 14: Agent receives result and responds to user
Key point: The invocation_id is how ADK knows to resume the paused execution instead of starting a new one.



📊 Section 5: Summary - Key Patterns for Advanced Tools
In this notebook, you implemented two powerful, production-ready patterns for extending your agent's capabilities beyond simple functions.

Pattern	When to Use It	Key ADK Components
MCP Integration	You need to connect to external, standardized services (like time, databases, or file systems) without writing custom integration code.	McpToolset
Long-Running Operations	You need to pause a workflow to wait for an external event, most commonly for human-in-the-loop approvals or long background tasks or for compliance/security checkpoints.	ToolContext, request_confirmation, App, ResumabilityConfig
🚀 Production Ready Concepts
You now understand how to build agents that:

🌐 Scale: Leverage community tools instead of building everything
⏳ Handle Time: Manage operations that span minutes, hours, or days
🔒 Ensure Compliance: Add human oversight to critical operations
🔄 Maintain State: Resume conversations exactly where they paused
Start Simple: Begin with custom tools → Add MCP services → Add long-running as needed